In [ ]:
!git clone https://github.com/Gsus01/NuweOracle-AnalisisDeSentimiento.git repo

In [28]:
!pip install datasets transformers

El nombre de archivo, el nombre de directorio o la sintaxis de la etiqueta del volumen no son correctos.


---

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("csv", data_files="repo/data/train.csv")
temp         = load_dataset("csv", data_files="repo/data/test.csv")
raw_datasets["test"] = temp["train"]
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
  return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
  columns=["attention_mask", "input_ids", "token_type_ids"],
  label_cols=["label"],
  shuffle=True,
  collate_fn=data_collator,
  batch_size=8,
)

# tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
#   columns=["attention_mask", "input_ids", "token_type_ids"],
#   label_cols=["label"],
#   shuffle=False,
#   collate_fn=data_collator,
#   batch_size=8,
# )

c:\Users\3arci\miniconda3\envs\oracle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (C:/Users/3arci/.cache/huggingface/datasets/csv/default-c86a7e28974effee/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 500.22it/s]
Found cached dataset csv (C:/Users/3arci/.cache/huggingface/datasets/csv/default-22b7c1d085192d86/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 998.88it/s]
Loading cached processed dataset at C:\Users\3arci\.cache\huggingface\datasets\csv\default-c86a7e28974effee\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-721e420b36bcdffa.arrow
Loading cached processed dataset at C:\Users\3arci\.cache\huggingface\datasets\csv\default-22b7c1

In [2]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# from tensorflow.keras.losses import SparseCategoricalCrossentropy

# model.compile(
#   optimizer="adam",
#   loss=SparseCategoricalCrossentropy(from_logits=True),
#   metrics=["accuracy"],
# )
# model.fit(
#   tf_train_dataset,
#   # validation_data=tf_validation_dataset,
# )

In [3]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
  initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [4]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.fit(tf_train_dataset, epochs=3)

Epoch 1/3


: 

: 

---

In [4]:
model = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [15]:
predictions = model(list(test['text']))

In [16]:
target = {}
for idx, pred in enumerate(predictions):
  target[str(idx)] = 1 if pred['label'] == 'POSITIVE' else 0
predictions = { 'target': target }

In [23]:
import json

with open('predictions.json', 'w') as f:
  json.dump(predictions, f, indent=2)